In [ ]:
from urllib.request import parse_http_list
from IPython.core.interactiveshell import available_events
from traitlets.config.loader import Undefined
from os import truncate
import random

class Part():

    #Contrutor de uma parte
    def __init__(self, name: str, attack_level=0, defense_level=0, energy_consumption=0):
        self.name = name
        self.attack_level = attack_level
        self.defense_level = defense_level
        self.energy_consumption = energy_consumption
        self.color = ""
        self.colors = {
                "Red": '\033[31m',
                "Green": '\033[32m',
                "Blue": '\033[34m',
                "Cyan": '\033[36m',
                "Pink": '\033[35m',
                "Yellow": '\033[33m',
                "Black": '\033[30m',
                "White": '\033[37m',
                "Reset": '\033[0;0m'
        }
      
    #Pega as informações daquela parte
    def get_status_dict(self, reset):
        formatted_name = self.name.replace(" ", "_").lower()

        if self.is_available():
            self.color = color = self.colors["Green"]
        else:
            self.color = self.colors["Red"]
        
        return {
            f"{formatted_name}_name": self.name.lower(),
            f"{formatted_name}_status": self.is_available(),
            f"{formatted_name}_attack": self.attack_level,
            f"{formatted_name}_defense": self.defense_level,
            f"{formatted_name}_energy_consump": self.energy_consumption,
            f"{formatted_name}_color": self.color,
            f"{formatted_name}_reset": self.colors[reset],
        }
        
    #Reduz a defesa de alguma parte que sofreu dano
    def reduce_defense(self, attack_level):
        self.defense_level = self.defense_level - attack_level
        if self.defense_level <= 0:
            self.defense_level = 0

    #Verifica se a parte está disponível ainda
    def is_available(self):
        name = self.name.replace(" ", "_").lower()
        if name == "weapon":
            return True
        
        return self.defense_level > 0

class Robot():

    #Construtor do Robo
    def __init__(self, name):
        self.name = name
        self.energy = 100
        self.parts = [
            Part("Head", attack_level=2, defense_level=20, energy_consumption=5),
            Part("Weapon", attack_level=20, defense_level=0, energy_consumption=20),
            Part("Left Arm", attack_level=10, defense_level=20, energy_consumption=15),
            Part("Right Arm", attack_level=10, defense_level=20, energy_consumption=15),
            Part("Left Leg", attack_level=5, defense_level=20, energy_consumption=10),
            Part("Right Leg", attack_level=5, defense_level=20, energy_consumption=10),
            Part("shield", attack_level=0, defense_level=2, energy_consumption=10),
            Part("sword", attack_level=15, defense_level=2, energy_consumption=10),
        ]
        self.colors = {
                "Red": '\033[31m',
                "Green": '\033[32m',
                "Blue": '\033[34m',
                "Cyan": '\033[36m',
                "Pink": '\033[35m',
                "Yellow": '\033[33m',
                "Black": '\033[30m',
                "White": '\033[37m',
                "Reset": '\033[0;0m'
        }
        self.shield = False
        self.countShield = 0
        self.sword = False
        self.countSword = 0
    
    #Atacando alguém
    def attack(self, enemy_robot, part_to_use, part_to_attack):
        if part_to_use == 7:
                self.parts[part_to_use].reduce_defense(1)
                
        if part_to_attack == 6:
            enemy_robot.shield = False
            enemy_robot.parts[part_to_attack].reduce_defense(1)
        else:
            enemy_robot.parts[part_to_attack].reduce_defense(self.parts[part_to_use].attack_level)

        self.energy -= self.parts[part_to_use].energy_consumption

    #Gera um equipamento super
    def set_super(self):
        supers = [
            Part("Armor", attack_level=0, defense_level=30, energy_consumption=0),
            Part("Letal Weapom", attack_level=15, defense_level=0, energy_consumption=10)
        ]
        super = random.choice(supers)
        self.parts["super"] = super
        print("You created me, so take this super part")
        super.print_status()
    
    #Mostra o robô, com o status como está a situação de cada parte
    def print_status(self, bot):
        robot_art = r"""
                                     |0: {head_name}
                                     |Is available: {head_status}
               --------------------> |Attack: {head_attack}                              
               |                     |Defense: {head_defense}
               |                     |Energy consumption: {head_energy_consump}
               |
               |                     |1: {weapon_name}
               |                     |Is available: {weapon_status}
    {weapon_color} ____ {weapon_reset}     |     {weapon_color}____  {weapon_reset}         |Attack: {weapon_attack}
   {weapon_color} |oooo| {head_color}  ____  {weapon_color} |oooo|{weapon_reset} -------> |Defense: {weapon_defense}
   {weapon_color} |oooo| {head_color} '    ' {weapon_color} |oooo|{weapon_reset}          |Energy consumption: {weapon_energy_consump}
   {weapon_color} |oooo| {head_color}/\_||_/\ {weapon_color}|oooo|{weapon_reset}        
   {weapon_color} `----'  {head_color}/ __ \  {weapon_color}`----' {weapon_reset}         |2: {left_arm_name}
    '/  |#|/\/__\/\|#|  \'           |Is available: {left_arm_status}
    /  \|#|| |/\| ||#|/  \           |Attack: {left_arm_attack}
   / \_/|_|| |/\| ||_|\_/ \          |Defense: {left_arm_defense}
  {left_arm_color} |_\/ {left_arm_reset}  O\=----=/O  {right_arm_color} \/_| {right_arm_reset}         |Energy consumption: {left_arm_energy_consump}
  {left_arm_color} <_> {left_arm_reset}    |=\__/=|    {right_arm_color} <_> {right_arm_reset}-------> |
  {left_arm_color} <_> {left_arm_reset}    |------|    {right_arm_color} <_> {right_arm_reset}         |3: {right_arm_name}
  {left_arm_color} | | {left_arm_reset} ___|======|___ {right_arm_color} | | {right_arm_reset}         |Is available: {right_arm_status}
 {left_arm_color}// \\{left_arm_reset} / |O|======|O| \ {right_arm_color}//\\ {right_arm_reset}        |Attack: {right_arm_attack}
 {left_arm_color} |  |{left_arm_reset} | |O+------+O| | {right_arm_color}|  | {right_arm_reset}        |Defense: {right_arm_defense}
 {left_arm_color} |\/|{left_arm_reset} \_+/        \+_/ {right_arm_color}|\/| {right_arm_reset}        |Energy consumption: {right_arm_energy_consump}
 {left_arm_color} \__/{left_leg_color} _|||     {right_leg_color}   |||_ {right_arm_color}\__/ {right_arm_reset}       
      {left_leg_color} | ||      {right_leg_color}  || | {right_leg_reset}             |4: {left_leg_name} 
      {left_leg_color}[==|]      {right_leg_color}  [|==] {right_leg_reset}            |Is available: {left_leg_status}
      {left_leg_color}[===]      {right_leg_color}  [===] {right_leg_reset}            |Attack: {left_leg_attack}
      {left_leg_color} >_<       {right_leg_color}   >_<  {right_leg_reset}            |Defense: {left_leg_defense}
      {left_leg_color}|| ||      {right_leg_color}  || || {right_leg_reset}            |Energy consumption: {left_leg_energy_consump}
      {left_leg_color}|| ||      {right_leg_color}  || ||{right_leg_reset} ----------> |
      {left_leg_color}|| ||      {right_leg_color}  || || {right_leg_reset}            |5: {right_leg_name}
   {left_leg_color} __|\_/|__    {right_leg_color}__|\_/|__  {right_leg_reset}         |Is available: {right_leg_status}
  {left_leg_color} /___n_n___\  {right_leg_color}/___n_n___\  {right_leg_reset}        |Attack: {right_leg_attack}
                                     |Defense: {right_leg_defense}
                                     |Energy consumption: {right_leg_energy_consump}


{shield_color}  _________________________  {shield_reset}
{shield_color} |<><><>     |  |    <><><>| {shield_reset}
{shield_color} |<>         |  |        <>| {shield_reset}
{shield_color} |           |  |          | {shield_reset}
{shield_color} |  (______ <\-/> ______)  | {shield_reset}
{shield_color} |  /_.-=-.\| " |/.-=-._\  | {shield_reset}
{shield_color} |   /_    \(o_o)/    _\   | {shield_reset}
{shield_color} |    /_  /\/ ^ \/\  _\    | {shield_reset}          |6: {shield_name}
{shield_color} |      \/ | / \ | \/      | {shield_reset}          |Is avaible: {shield_status}
{shield_color} |_______ /((( )))\ _______| {shield_reset}--------> |Attack: {shield_attack}
{shield_color} |      __\ \___/ /__      | {shield_reset}          |Uses: {shield_defense}
{shield_color} |--- (((---'   '---))) ---| {shield_reset}          |Energy consumption: {shield_energy_consump}
{shield_color} |           |  |          | {shield_reset}
{shield_color} |           |  |          | {shield_reset}
{shield_color} :           |  |          : {shield_reset}
{shield_color}  \<>        |  |       <>/  {shield_reset}
{shield_color}   \<>       |  |      <>/   {shield_reset}
{shield_color}    \<>      |  |     <>/    {shield_reset}
{shield_color}     `\<>    |  |   <>/'     {shield_reset}
{shield_color}       `\<>  |  |  <>/'      {shield_reset}
{shield_color}         `\<>|  |<>/'        {shield_reset}
{shield_color}           `-.  .-`          {shield_reset}
{shield_color}             '--'            {shield_reset}


{sword_color}          .         {sword_reset}
{sword_color}         / \        {sword_reset}
{sword_color}        /   \       {sword_reset}
{sword_color}       /  |  \      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}          |7: {sword_name}
{sword_color}       |  |  |      {sword_reset}          |Is avaible: {sword_status}
{sword_color}       |  |  |      {sword_reset} -------> |Attack: {sword_attack}
{sword_color}       |  |  |      {sword_reset}          |Uses: {sword_defense}
{sword_color}       |  |  |      {sword_reset}          |Energy consumption: {sword_energy_consump}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}       |  |  |      {sword_reset}
{sword_color}  ____/|  |  |\____ {sword_reset}
{sword_color} /____/\___/\____/ ){sword_reset}
{sword_color} (_______________)/ {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}       |/////|      {sword_reset}
{sword_color}      (_______)     {sword_reset}

"""

        color = self.returnColor(bot)
        str_robot = robot_art.format(**self.get_part_status(color))
        print("")
        print(self.colors[color])
        self.greet()
        self.print_energy()
        print(str_robot)
        print(self.colors["Reset"])
    
    #Printa um cumprimento, onde o robô diz oi
    def greet(self):
        print("Hello, my name is", self.name)
    
    #Mostra a quantidade de energia que o robô possui
    def print_energy(self):
        print("\nWe have", self.energy, "percent energy left")
    
    #Pega a situação de cada parte
    def get_part_status(self, color):
        part_status = {}
        for part in self.parts:
            status_dict = part.get_status_dict(color)
            part_status.update(status_dict)
        return part_status
        
    #Verifica se a parte está disponível
    def is_there_available_part(self):
        for part in self.parts:
            if part.is_available():
                return True
        return False
    
    #Verifica se o robô esta com energia
    def is_on(self):
        return self.energy >= 0
    
    def returnColor(self, num):
        if (num == 1):
            return "Cyan"
        else:
            return "Yellow"
    
    def activeShield(self):
        self.shield = True
        self.countShield += 1
    
    def shieldAvaible(self):
        if self.countShield < 2:
            return True
        else:
            self.parts[6].changeStatus()
            return False

    def activeSword(self):
        self.sword = True
        self.countSword += 1
    
    def swordAvaible(self):
        if self.countSword < 2:
            return True
        else:
            self.parts[7].changeStatus()
            return False

class Play():
    def __init__(self):
        self.name = ""
        self.player1 = ""
        self.player2 = ""
        self.parts = [
            "weapon",
            "head",
            "left_arm",
            "right_arm",
            "left_leg",
            "right_leg",
            "sword"
        ]

        self.parts2 = [
            "head",
            "left_arm",
            "right_arm",
            "left_leg",
            "right_leg"
        ]
    
    def receiveName(self, bot):
        self.name = input(f"Please insert name of the robot {bot}: ")
    

    def isDied(self, bot):
        part_status = self.getPartStatus(bot)
        avaibleParts = self.verifyParts(bot, self.parts2)
        
        if (len(avaibleParts) == 0):
            return True
        else:
            return False
    

    def getPartStatus(self, bot):
        if bot == 1:
            part_status = self.player1.get_part_status("Cyan")
        else:
            part_status = self.player2.get_part_status("Yellow")

        return part_status


    def verifyParts(self, bot, parts):
        partStatus = self.getPartStatus(bot)

        avaibleParts = []

        for part in parts:
            if (partStatus[f"{part}_status"]):
                avaibleParts.append(partStatus[f"{part}_name"])

        return avaibleParts


    def listPossibilities(self, bot, parts):
        avaibleParts = self.verifyParts(bot, parts)

        for part in avaibleParts:
            print( part)

    def respAttack(self, bot):
        verify = True
        resp = ""

        while(verify):
            resp = input("\nWhich part do you want to use to attack? ").lower().strip().replace(" ", "_").replace("__", "_")
            print(resp)

            if resp not in self.parts:
                verify = True
                print("Please insert a valid value. ")
            else:
                verify = False
        
        return self.findIndex(resp)
    
    def respDefense(self, bot):
        verify = True
        resp = ""
        parts = self.verifyParts(bot, self.parts2)

        while(verify):
            resp = input("\nWhich part do you want to attack on your opponent? ").lower().strip().replace(" ", "_").replace("__", "_")

            if resp not in self.parts:
                verify = True
                print("Please insert a valid value. ")
            else:
                verify = False
        
        return self.findIndex(resp)
    
    def findIndex(self, resp):
        if resp == "head":
            return 0
        elif resp == "weapon":
            return 1
        elif resp == "left_arm":
            return 2
        elif resp == "right_arm":
            return 3
        elif resp == "left_leg":
            return 4
        elif resp == "right_leg":
            return 5
        elif resp == "shield":
            return 6
        elif resp == "sword":
            return 7

    def attack(self, yourBot, enemyBot):
        if (yourBot == 1):
            self.player1.print_status(1)
        else:
            self.player2.print_status(2)

        print('''
Possible Forms to Attack
------------------------
''')
        self.listPossibilities(yourBot, self.parts)
        attack = self.respAttack(yourBot)

        if (enemyBot == 1):
            if self.player1.shield:
                self.player1.attack(self.player2, attack, 6)
                return
        elif self.player2.shield:
            self.player2.attack(self.player1, attack, 6)
            return

        if (enemyBot == 1):
            self.player1.print_status(1)
        else:
            self.player2.print_status(2)

        print('''
Parts Avaible to be Attacked
----------------------------
''')
        self.listPossibilities(enemyBot, self.parts2)
        defense = self.respDefense(enemyBot)

        if(yourBot == 1):
            self.player1.attack(self.player2, attack, defense)
        else:
            self.player2.attack(self.player1, attack, defense)
    
    def defense(self, bot):
        if(bot == 1):
            return self.player1.shieldIsOn()
        else:
            return self.player2.shieldIsOn()

    def optionsAvaible(self, bot):
        if(bot == 1):
            return self.player1.countShield < 2
        else:
            return self.player2.countShield < 2

    def respTurn(self):
        verifyResp = True
        resp = ""

        while(verifyResp):
            resp = input("\nWhat do you want to do? ").lower().strip()

            if resp in ("attack", "defense"):
                verifyResp = False
            else:
                print("Please insert a valid value. ")
        
        return resp

    def turn(self, yourBot, enemyBot):
        print(f'''
-------------------------
It's Turn of the Player {yourBot}
-------------------------
''')
        if self.optionsAvaible(yourBot):
            print("Options: \n \nAttack \nDefense")
            resp = self.respTurn()
        else:
            resp = "attack"

        if resp == "attack":
            self.attack(yourBot, enemyBot)
        else:
            self.player1.activeShield()
            self.player1.energy -= self.player1.parts[6].energy_consumption

    def initial(self):
        print('''
****************************************************
            Wellcome to Robot War!
****************************************************

What is Robot War?
------------------
Robot War is a turn-based game, where two players face each other, having the possibility to attack and defend.


Instructions
------------
At the beginning of the game, players will be able to choose the name of their robots, after that, the game will start. 
The first player will have two options on his screen: 'attack' or 'defense'. In the 'defense' option, the player will 
raise a shield and pass the turn, thus protecting himself from the opponent's next attack. 
In the 'attack' option, the player will have to make two choices: the part of the body he wants to use to attack the 
opponent and the part of the opponent's body he wants to attack. 
The game ends when one of the two loses all members or runs out of energy.

Note:. The shield and sword can only be used twice.

Good Game and Have Fun!
''')

    def run(self):
        self.initial()

        self.receiveName(1)
        self.player1 = Robot(self.name)
        self.player1.print_status(1)

        self.receiveName(2)
        self.player2 = Robot(self.name)
        self.player2.print_status(2)

        verify = True

        countTurn = 0

        while(verify):
            if self.isDied(1):
                verify = False
                print(f"Robot {self.player2.name} Wins")
                break
            elif self.isDied(2):
                verify = False
                print(f"Robot {self.player1.name} Wins")
                break
            
            if countTurn == 0:
                self.turn(1, 2)
                countTurn = 1
            else:
                self.turn(2, 1)
                countTurn = 0
            
            self.player1.print_status(1)
            self.player2.print_status(2)

game = Play()
game.run()


****************************************************
            Wellcome to Robot War!
****************************************************

What is Robot War?
------------------
Robot War is a turn-based game, where two players face each other, having the possibility to attack and defend.


Instructions
------------
At the beginning of the game, players will be able to choose the name of their robots, after that, the game will start. 
The first player will have two options on his screen: 'attack' or 'defense'. In the 'defense' option, the player will 
raise a shield and pass the turn, thus protecting himself from the opponent's next attack. 
In the 'attack' option, the player will have to make two choices: the part of the body he wants to use to attack the 
opponent and the part of the opponent's body he wants to attack. 
The game ends when one of the two loses all members or runs out of energy.

Note:. The shield and sword can only be used twice.

Good Game and Have Fun!

